In [1]:
import pandas as pd
import sqlite3 as sl

In [2]:
#create database object
conn = sl.connect('NYT_covid.db')
c = conn.cursor()

In [3]:
# this will give you the names of the tables that were created and are currently in the db.
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print (name[0])

deaths
mask_use
counties
states
us


In [4]:
df = pd.read_sql('''select * from counties''',conn)
df

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0
...,...,...,...,...,...,...
515057,2020-09-08,Sweetwater,Wyoming,56037.0,314,2
515058,2020-09-08,Teton,Wyoming,56039.0,450,1
515059,2020-09-08,Uinta,Wyoming,56041.0,309,2
515060,2020-09-08,Washakie,Wyoming,56043.0,110,6
